In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import pickle
! pip install --q blingfire spacy transformers sentence-transformers umap-learn hdbscan stop-words
import re
import itertools
import collections

In [ ]:
!pip install cuml-cu11 --extra-index-url=https://pypi.nvidia.com
import cuml

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [ ]:
import pandas as pd
from google.colab import drive
df = pd.read_csv(r'/content/drive/MyDrive/patent project/clean_data_with_enrich.csv')

<ipython-input-38-0d1fabb3f66d>:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'/content/drive/MyDrive/patent project/clean_data_with_enrich.csv')


In [ ]:
df = df[df['one_if_small']==1]
df = df[df['Patent_Target']!='pending']
df = df[['application_number','Gender_Target','Patent_Target','abstract']]

In [ ]:
#cleaning function packages
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
stop = stopwords.words('english')
wl = WordNetLemmatizer()

In [ ]:

def clean_data(text):
  word_tokens = word_tokenize(text) #split into words
  clean_words = []
  for word  in word_tokens:
    if word.isalpha() and word.lower() not in stop:
        clean_word = wl.lemmatize(word.lower())
        clean_words.append(clean_word)
  clean_text = " ".join(clean_words)
  return clean_text


In [ ]:
df.head()

,application_number,Gender_Target,Patent_Target,abstract
2,13261421,Male,patented,The present invention provides adjuvant compos...
3,13261486,Male,patented,The disclosed invention relates to solar-therm...
4,13261494,Male,patented,A lamp assembly ( 500 ) may include a linear l...
5,13261526,Male,abandoned,One embodiment includes a launch vessel defini...
6,13261566,Male,abandoned,The present invention relates to a bottle cap ...


In [ ]:
#get sentences from abstracts
sentence_items = []
for i in range(df.shape[0]):
  text = df.iloc[i,3]
  document_sentences = sent_tokenize(text)
  for s in document_sentences:
    sentence_items.append((s,df.iloc[i,0]))

In [ ]:
df_sentences = pd.DataFrame(sentence_items, columns=['original_sentence', 'document_id'])

In [ ]:
#clean
df_sentences['original_sentence'] = df_sentences['original_sentence'].apply(clean_data)


In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
import tensorflow as tf


In [ ]:
#model = SentenceTransformer('all-MiniLM-L6-v2')

model = SentenceTransformer('all-mpnet-base-v2')
with tf.device('/device:GPU:0'):
  embeddings_df = model.encode(df_sentences['original_sentence'].values, show_progress_bar=True)

Batches:   0%|          | 0/16236 [00:00<?, ?it/s]

In [ ]:
import pickle
fHandle = open(r'/content/drive/MyDrive/patent project/patent_embed.pickle','wb')
pickle.dump(embeddings_df, fHandle,protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
fHandle.close()

In [ ]:
#load pickle
pickle_file = open(r'/content/drive/MyDrive/patent project/patent_embed.pickle', 'rb')
embeddings_df = pickle.load(pickle_file)

In [ ]:
#import umap
from cuml import UMAP

In [ ]:
with tf.device('/device:GPU:0'):
  embeddings_reduced = cuml.UMAP(
      n_neighbors=15,
      n_components=5,
      metric='cosine',
  ).fit_transform(embeddings_df)

In [ ]:
umap_file = open(r'/content/drive/MyDrive/patent project/patent_umap.pickle','wb')
pickle.dump(embeddings_reduced, umap_file,protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
umap_file.close()

In [ ]:
import pickle
pickle_file = open(r'/content/drive/MyDrive/patent project/patent_umap.pickle', 'rb')
embeddings_reduced = pickle.load(pickle_file)

In [ ]:
#import hdbscan

In [ ]:
with tf.device('/device:GPU:0'):
  cluster = cuml.cluster.hdbscan.HDBSCAN(
      min_cluster_size=110,
      min_samples = 1,
      metric='euclidean',
      cluster_selection_method='eom').fit(embeddings_reduced)

In [ ]:
df_sentences['cluster_id'] = cluster.labels_

In [ ]:
df_sentences_clustered = df_sentences[df_sentences['cluster_id'] != -1]

In [ ]:
df_sentences_clustered.to_csv(r'/content/drive/MyDrive/patent project/patented_df_cluster.csv',index=False)

In [ ]:
df_sentences_clustered['document_id'].nunique()

99088

In [ ]:
df_sentences['document_id'].nunique()

129068

In [ ]:
df_sentences_clustered['cluster_id'].nunique()

499

In [ ]:
df_sentences_clustered.head()

,original_sentence,document_id,cluster_id
2,disclosed invention relates receiver tube heat...,13261486,371
5,lamp assembly may include linear array reflect...,13261494,336
6,reflecting surface may shaped cause distributi...,13261494,336
7,reflecting surface may part reflector may incl...,13261494,336
12,present invention relates bottle cap comprisin...,13261566,362


In [ ]:
df_topics = df_sentences_clustered.groupby(['cluster_id'], as_index = False).agg({'original_sentence': ' '.join}).rename(columns={'original_sentence': 'cluster_sentences'})

In [ ]:
df_topics.to_csv(r'/content/drive/MyDrive/patent project/patented_df_topics.csv',index=False)

**Analysis**

In [ ]:
df_sentences_clustered = pd.read_csv(r'/content/drive/MyDrive/patent project/patented_df_cluster.csv')

In [ ]:
#get gender and patened status for each documnet
df = df[['application_number','Gender_Target','Patent_Target']]

In [ ]:
df[(df['one_if_small']==1) & (df['Patent_Target']!='pending')]['Patent_Target'].value_counts(normalize=True)

patented     0.59526
abandoned    0.40474
Name: Patent_Target, dtype: float64

In [ ]:
df[(df['one_if_small']==1) & (df['Patent_Target']!='pending')]['Gender_Target'].value_counts(normalize=True)

Male      0.871277
Female    0.128723
Name: Gender_Target, dtype: float64

In [ ]:
df_sentences_clustered = df_sentences_clustered.merge(df,how='inner',left_on='document_id',right_on='application_number')

In [ ]:
test_gender = pd.pivot_table(df_sentences_clustered,index='cluster_id',columns='Gender_Target',aggfunc='count',values='application_number').reset_index()

In [ ]:
test_gender['female_ratio'] = test_gender.apply(lambda x: x['Female']/(x['Female']+x['Male']),axis=1)

In [ ]:
test_reject = pd.pivot_table(df_sentences_clustered,index='cluster_id',columns='Patent_Target',aggfunc='count',values='application_number').reset_index()

In [ ]:
test_reject['reject_ratio'] = test_reject.apply(lambda x: x['abandoned']/(x['abandoned']+x['patented']),axis=1)

In [ ]:
reject_clusters = test_reject[test_reject['reject_ratio']>=0.4]['cluster_id'].to_list()

In [ ]:
len(reject_clusters)

210

In [ ]:
female_clusters = test_gender[test_gender['female_ratio'] >= 0.5]['cluster_id'].to_list()

In [ ]:
len(female_clusters)

13

In [ ]:
counter = 0
for i in female_clusters:
  if i in reject_clusters:
    counter+=1
print(counter)

13


In [ ]:
import pandas as pd
!pip install keybert
from keybert import KeyBERT


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.2 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23776 sha256=ced5be73ac83df3d8614d2e0abc3e09db28772ef167f2bfd7c0396b44b523d5e
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=7b453596b3e2331ac5852906a59be3170d51a2da29db1e70b734f7446bddc862
 

In [ ]:
df_topics = pd.read_csv(r'/content/drive/MyDrive/patent project/patented_df_topics.csv')

In [ ]:
kw_model = KeyBERT()


In [ ]:
df_topics['topics'] = 'topic'

In [ ]:
df_topics.shape[0]

499

In [ ]:
for i in range(df_topics.shape[0]):
  df_topics.iat[i,2] = kw_model.extract_keywords(df_topics.iat[i,1], keyphrase_ngram_range=(1, 1), stop_words=None)

In [ ]:
df_topics['topics'] = df_topics['topics'].apply(lambda x: x[:5])

In [ ]:
df_topics.to_csv(r'/content/drive/MyDrive/patent project/cluster_topics.csv',index=False)